In [57]:
from bs4 import BeautifulSoup 
from selenium import webdriver
import urllib3
import time 
import random
import requests
import re
import pandas as pd 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [58]:
url  = 'https://hellowork.com/fr-fr/emploi/recherche.html?k=data&k_autocomplete=&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&msa=&cod=all&d=all&c_idesegal=#47413184'
driver_path = print(ChromeDriverManager().install())
driver = webdriver.Chrome(driver_path)
driver.get(url)

# Use find_elements instead of find_element in order to specify this as a list instead of a string so we will be able to use the command find_element later on
job_postings_elements = driver.find_elements(By.CLASS_NAME,"crushed")
print(job_postings_elements)

C:\Users\oumis\.wdm\drivers\chromedriver\win64\122.0.6261.128\chromedriver-win32/chromedriver.exe
[<selenium.webdriver.remote.webelement.WebElement (session="4f8c23227eaf81ce3deb28d61f6ccad4", element="f.FBB825C8E8EBB494BE9BC42EAAC41645.d.F914EFF2FEFFDC9294BC14ABFECF8097.e.358")>, <selenium.webdriver.remote.webelement.WebElement (session="4f8c23227eaf81ce3deb28d61f6ccad4", element="f.FBB825C8E8EBB494BE9BC42EAAC41645.d.F914EFF2FEFFDC9294BC14ABFECF8097.e.359")>, <selenium.webdriver.remote.webelement.WebElement (session="4f8c23227eaf81ce3deb28d61f6ccad4", element="f.FBB825C8E8EBB494BE9BC42EAAC41645.d.F914EFF2FEFFDC9294BC14ABFECF8097.e.383")>, <selenium.webdriver.remote.webelement.WebElement (session="4f8c23227eaf81ce3deb28d61f6ccad4", element="f.FBB825C8E8EBB494BE9BC42EAAC41645.d.F914EFF2FEFFDC9294BC14ABFECF8097.e.384")>, <selenium.webdriver.remote.webelement.WebElement (session="4f8c23227eaf81ce3deb28d61f6ccad4", element="f.FBB825C8E8EBB494BE9BC42EAAC41645.d.F914EFF2FEFFDC9294BC14ABFECF8

In [59]:
# This is a function to check the existence of a certain value in the job posting.
# In our case, we will use it in the unit below to check whether the job post discloses a salary or not
def check_existence(value,s):
    try:
        s.find_element(By.CLASS_NAME, value)
        return True
    except:
        return False

In [60]:
job_list = []
for job in job_postings_elements:
    try:
        o = {}
# Some of the elements are hidden with CSS (display: none, visibility: hidden), hence they may not be accessible via .text.
# We use get_attribute("innerText") instead of text to resolve the issue
       
        # All job titles end with H/F. Thus, we have to rmeove it for cleaner a table
        title_text = job.find_element(By.CLASS_NAME, 'tw-text-2xlOld').get_attribute("innerText")
        o['title'] = title_text.replace(' H/F', '') # To remove H/F
        
        o['company'] = job.find_element(By.CLASS_NAME,'tw-mr-2').get_attribute("innerText")
        o['contract'] = job.find_element(By.CLASS_NAME,'tw-w-max').get_attribute("innerText")
        
        
        # Location contains both city and postal code. To split the location text into city and postal code
        location_text = job.find_element(By.CLASS_NAME, 'tw-text-ellipsis').get_attribute("innerText")
        city, postal_code = location_text.split(' - ')
        o['city'] = city.strip() 
        o['postal_code'] = postal_code.strip()

        
        #Some jobs disclose the salary while others don't. 
        salary_element = job.find_elements(By.CLASS_NAME, 'tw-bg-jobsCandidacyLightest')
        o['salary'] = salary_element[0].get_attribute("innerText") if salary_element else "Not disclosed"

        # Some jobs allow partial remote work while others dont. One important note is that the element for remote work is not specified 
        # as a class, but rather as a Cypress. Thereore, we use the CSS_SELECTOR instead of CLASS_NAME
        remote_work_element = job.find_elements(By.CSS_SELECTOR, 'span[data-cy="teleworkInfo"]')
        o['remote_work'] = 'Allowed' if remote_work_element else 'Not Allowed'

        
        job_list.append(o)
    except Exception as e:
        print("Error:", e)
      
# Transforming our list into a dataframe 
df = pd.DataFrame(job_list)
# Remo
df = df.drop_duplicates()
df = df.reset_index(drop=True)
df

Error: not enough values to unpack (expected 2, got 1)


,title,company,contract,city,postal_code,salary,remote_work
0,Consultant Développeur Data,Harry Hope,CDI,Strasbourg,67,35 000 - 45 000 EUR par an,Allowed
1,Opérateur - Technicien Data Référentiels,CREDIT INDUSTRIEL ET COMMERCIAL,CDI,Strasbourg,67,30 000 - 35 000 EUR par an,Allowed
2,Consultant Senior Data et Bi,Metsys,CDI,Strasbourg,67,Not disclosed,Allowed
3,Business Manager - Digital & Data,Astek,CDI,Strasbourg,67,Not disclosed,Allowed
4,Consultant Senior Data et Bi,Metsys,CDI,Strasbourg,67,40 000 - 60 000 EUR par an,Allowed
5,Data Manager,Crédit Foncier et Communal d'Alsace et de Lorr...,CDI,Strasbourg,67,Not disclosed,Not Allowed
6,Data Scientist,Viveris.,CDI,Strasbourg,67,35 000 - 50 000 EUR par an,Allowed
7,Data Manager,Hays,CDI,Strasbourg,67,Not disclosed,Not Allowed
8,Data Analyst,Akkodis,CDI,Strasbourg,67,Not disclosed,Not Allowed
9,Data Scientist,Extia,CDI,Strasbourg,67,Not disclosed,Allowed


In [7]:
r = requests.get("https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=1&mode=pagination")
print(r.status_code)

403


In [61]:
def get_all_pages():
    page_number = 1
    
    for i in range(1, 11):
        i = f"https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p={page_number}&mode=pagination"
        print(i)
        page_number += 1
        
get_all_pages()



https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=1&mode=pagination
https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=2&mode=pagination
https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=3&mode=pagination
https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=4&mode=pagination
https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=5&mode=pagination
https://ww

In [37]:
def get_all_pages():
    urls = []
    page_number = 1
    
    for i in range(1, 11):
        i = f"https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p={page_number}&mode=pagination"
        page_number += 1
        urls.append(i)
    
    return urls

def parse_attorney():
    r = requests.get("https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=1&mode=pagination")
    soup = BeautifulSoup(r.content, "html.parser")
    
    print(soup)
    
parse_attorney()

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
<hr/><center>Microsoft-Azure-Application-Gateway/v2</center>
</body>
</html>



In [62]:
import pandas as pd

def get_all_pages():
    page_number = 1
    urls = []
    
    for i in range(1, 11):
        i = f"https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p={page_number}&mode=pagination"
        page_number += 1
        urls.append(i)
        
    return urls
        
def parse_attorney(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    jobs = soup.find_all('div', class_="offer--maininfo md:!tw-w-11/12")
    
    job_list = []
    for job in jobs:
        try:
            o = {}

            title_text = job.find_element(By.CLASS_NAME, 'tw-text-2xlOld').get_attribute("innerText")
            o['title'] = title_text.replace(' H/F', '')

            o['company'] = job.find_element(By.CLASS_NAME,'tw-mr-2').get_attribute("innerText")
            o['contract'] = job.find_element(By.CLASS_NAME,'tw-w-max').get_attribute("innerText")

            location_text = job.find_element(By.CLASS_NAME, 'tw-text-ellipsis').get_attribute("innerText")
            city, postal_code = location_text.split(' - ')
            o['city'] = city.strip() 
            o['postal_code'] = postal_code.strip()

            salary_element = job.find_elements(By.CLASS_NAME, 'tw-bg-jobsCandidacyLightest')
            o['salary'] = salary_element[0].get_attribute("innerText") if salary_element else "Not disclosed"

            remote_work_element = job.find_elements(By.CSS_SELECTOR, 'span[data-cy="teleworkInfo"]')
            o['remote_work'] = 'Allowed' if remote_work_element else 'Not Allowed'

            job_list.append(o)
            
        except Exception as e:
            print("Error:", e)
            
    return job_list
        
def parse_all_attorneys():
    all_jobs = []
    pages = get_all_pages()
    for page in pages:
        jobs_on_page = parse_attorney(url=page)
        all_jobs.extend(jobs_on_page)
        print(f"Scraped {len(jobs_on_page)} jobs from {page}")
        
    return all_jobs

# Appel de la fonction pour obtenir tous les emplois de toutes les pages
all_jobs = parse_all_attorneys()

# Création d'un DataFrame à partir de la liste de dictionnaires contenant les emplois
df = pd.DataFrame(all_jobs)

# Affichage du DataFrame
print(df)


Scraped 0 jobs from https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=1&mode=pagination
Scraped 0 jobs from https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=2&mode=pagination
Scraped 0 jobs from https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=3&mode=pagination
Scraped 0 jobs from https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2Fwww.rj.com%2Fcommun%2Flocalite%2Fcommune%2F67482&ray=50&cod=all&d=all&p=4&mode=pagination
Scraped 0 jobs from https://www.hellowork.com/fr-fr/emploi/recherche.html?k=data&l=Strasbourg+67000&l_autocomplete=http%3A%2F%2F